<a href="https://colab.research.google.com/github/lucila-03/Acidente/blob/main/Acidente.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MVP para Análise de Informações do INSS
## Lucila da Conceição Castello Branco


O que não está detalhado ou pode ser melhorado neste notebook para ficar como esperamos para o MVP:
- Blocos de texto que expliquem textualmente cada etapa e cada decisão do seu código, contando uma história completa e compreensível, do início ao fim;
- Boas práticas de codificação;
- Após cada gráfico, escrever 1 parágrafo resumindo os principais achados, analisando os resultados e levantando eventuais pontos de atenção.

## 1. Definição do Problema *

O dataset usado neste projeto será um arquivo do governo com informações da Comunicação de Acidentes de Trabalho (CAT), são somente as informações que governo disponibiliza, o arquivo tem somente informações relativas aos de Janeiro e Fevereiro de 2022. O objetivo deste projeto é conhecer os tipos de acidentados em janeiro e fevereiro de 2022 que estão no INSS e conhecer os acidentes mais recorrentes. Este dataset é um subconjunto do dataset original, o original contem dados trimestrais.
    O dataset apresenta em diversos atributos relacionados a dados médicos e uma variável de classe binária (0 ou 1). As variáveis ​​preditoras incluem o número de gestações que a paciente teve, seu IMC, nível de insulina, idade e assim por diante.
    Para mais detalhes sobre este dataset, consulte: https://www.kaggle.com/uciml/pima-indians-diabetes-database


**Informações sobre os atributos:**
1. **preg** - Número de vezes grávida 
2. **plas** - Concentração de glicose no plasma
3. **pres** - Pressão sanguínea diastólica (mm Hg) 
4. **skin** - Espessura da dobra da pele do tríceps (mm) 
5. **test** - Insulina sérica de 2 horas (mu U/ml) 
6. **mass** - Índice de massa corporal (peso em kg/(altura em m)^2) 
7. **pedi** - Função de linhagem de diabetes (uma função que pontua a probabilidade de diabetes com base no histórico familiar) 
8. **age** - Idade (anos) 
9. **class** - Variável de classe (0 ou 1) 

In [ ]:
# Importação dos pacotes
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
# import missingno as ms # para tratamento de missings
# from matplotlib import cm
# from pandas import set_option
# from pandas.plotting import scatter_matrix
# from sklearn.preprocessing import StandardScaler
# from sklearn.preprocessing import MinMaxScaler
# from sklearn.model_selection import train_test_split
# from sklearn.model_selection import KFold
# from sklearn.model_selection import StratifiedKFold
# from sklearn.model_selection import cross_val_score
# from sklearn.model_selection import GridSearchCV
# from sklearn.metrics import classification_report
# from sklearn.metrics import confusion_matrix
# from sklearn.metrics import accuracy_score
# from sklearn.pipeline import Pipeline
# from sklearn.linear_model import LogisticRegression
# from sklearn.tree import DecisionTreeClassifier
# from sklearn.neighbors import KNeighborsClassifier
# from sklearn.naive_bayes import GaussianNB
# from sklearn.svm import SVC
# from sklearn.ensemble import BaggingClassifier
# from sklearn.ensemble import RandomForestClassifier
# from sklearn.ensemble import ExtraTreesClassifier
# from sklearn.ensemble import VotingClassifier
# from sklearn.ensemble import AdaBoostClassifier
# from sklearn.ensemble import GradientBoostingClassifier

In [ ]:
# configuração para não exibir os warnings
# import warnings
# warnings.filterwarnings("ignore")

## 2. Carga de Dados

In [ ]:
# Carrega arquivo csv usando Pandas usando uma URL

# Importação do dataset

url = "https://raw.githubusercontent.com/lucila-03/mvp/main/D.SDA.PDA.005.CAT.20220102.csv"
dataset = pd.read_csv(url, delimiter = ";", encoding='ISO-8859-1' )


## 3. Análise de Dados

### 3.1. Estatísticas Descritivas

Vamos iniciar examinando as dimensões do dataset, suas informações e alguns exemplos de linhas.

In [ ]:
# Mostra as dimensões do dataset
dataset.shape

(40852, 20)

In [ ]:
# Mostra as informações do dataset
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 40852 entries, 0 to 40851
Data columns (total 20 columns):
 #   Column                      Non-Null Count  Dtype 
---  ------                      --------------  ----- 
 0   Agente  Causador  Acidente  40852 non-null  object
 1   CBO                         40852 non-null  object
 2   CID-10                      40852 non-null  object
 3   CNAE2.0 Empregador          40852 non-null  object
 4   Emitente CAT                40852 non-null  object
 5   Espécie do benefício        40852 non-null  object
 6   Filiação Segurado           40852 non-null  object
 7   Indica Óbito Acidente       40852 non-null  object
 8   Munic Empr                  40852 non-null  object
 9   Natureza da Lesão           40852 non-null  object
 10  Parte Corpo Atingida        40852 non-null  object
 11  Sexo                        40852 non-null  object
 12  Tipo do Acidente            40852 non-null  object
 13  UF  Munic.  Acidente        40852 non-null  ob

É sempre importante verificar o tipo do atributos do dataset, pois pode ser necessário realizar conversões, com base nas informações do dataset, foi identificado que algumas colunas precisam mudar de atributo.

In [ ]:
from pandas.core.dtypes.common import is_int64_dtype
# Alterar atributo para data
#data = datetime.strptime('Sep 22 2021', '%b %d %Y')
#dataset['Data Acidente'] = datetime.strptime(dataset['Data Acidente'], '%b %d %Y' )


Após a solicitação de mudança precisamos verificar se ficou certo.

In [ ]:
 # Verifica o tipo de dataset de cada atributo
dataset.dtypes

Agente  Causador  Acidente    object
CBO                           object
CID-10                        object
CNAE2.0 Empregador            object
Emitente CAT                  object
Espécie do benefício          object
Filiação Segurado             object
Indica Óbito Acidente         object
Munic Empr                    object
Natureza da Lesão             object
Parte Corpo Atingida          object
Sexo                          object
Tipo do Acidente              object
UF  Munic.  Acidente          object
UF Munic. Empregador          object
Mês Acidente                  object
Data Despacho Benefício       object
Data Acidente                 object
Data Nascimento               object
Data Emissão CAT              object
dtype: object

In [ ]:
# Mostra as 5 primeiras linhas do dataset
dataset.head(5)

,Agente Causador Acidente,CBO,CID-10,CNAE2.0 Empregador,Emitente CAT,Espécie do benefício,Filiação Segurado,Indica Óbito Acidente,Munic Empr,Natureza da Lesão,Parte Corpo Atingida,Sexo,Tipo do Acidente,UF Munic. Acidente,UF Munic. Empregador,Mês Acidente,Data Despacho Benefício,Data Acidente,Data Nascimento,Data Emissão CAT
0,{ñ class},515105-Agente Comunitário de Saúde,B34.2 Infecc p/Coronavirus Ne,Atividades de Atencao Ambulatorial Executadas,Empregador,Pa,Empregado,Não,354780-Santo André-Sp,Doenca Contagiosa ou Infecciosa (Tuberculos,Aparelho Respiratorio,Feminino,Doença,Maranhão,São Paulo,2022/01,0000/00,20/01/2022,02/08/1970,01/03/2022
1,Impacto de Pes. Contra Objeto em Movimento,{ñ class},S90.0 Contusao do Tornozelo,Comercio Atacadista de Produtos Alimenticios,Empregador,Pa,Empregado,Não,330100-Campos dos Goytacazes,Luxacao,Perna (Entre O Tornozelo e a Pelvis),Masculino,Típico,Maranhão,São Paulo,2022/01,0000/00,28/01/2022,25/07/1995,01/03/2022
2,{ñ class},223505-Enfermeiro,B34.2 Infecc p/Coronavirus Ne,Atividades de Atencao Ambulatorial Executadas,Empregador,Pa,Empregado,Não,354780-Santo André-Sp,Doenca Contagiosa ou Infecciosa (Tuberculos,Aparelho Respiratorio,Feminino,Doença,Maranhão,São Paulo,2022/01,0000/00,18/01/2022,13/05/1982,01/03/2022
3,Passarela ou Plataforma Permanentes - Superfi,951105-Eletricista de Manut. Eletroeletrônica,S12.2 Frat de Outr Vertebras Cervicais Espec,Desdobramento de Madeira,Empregador,Pa,Empregado,Não,421500-Rio Negrinho,Fratura,"Dorso (Inclusive Musculos Dorsais, Coluna e M",Masculino,Típico,{ñ class},Santa Catarina,2022/01,0000/00,28/01/2022,21/03/1990,01/03/2022
4,{ñ class},{ñ class},B34.2 Infecc p/Coronavirus Ne,Atividades de Atencao Ambulatorial Executadas,Empregador,Pa,Empregado,Não,354780-Santo André-Sp,Doenca Contagiosa ou Infecciosa (Tuberculos,Aparelho Respiratorio,Feminino,Doença,Maranhão,São Paulo,2022/01,0000/00,20/01/2022,16/02/1965,01/03/2022


In [ ]:
# Mostra as 5 últimas linhas do dataset
dataset.tail(5)

,Agente Causador Acidente,CBO,CID-10,CNAE2.0 Empregador,Emitente CAT,Espécie do benefício,Filiação Segurado,Indica Óbito Acidente,Munic Empr,Natureza da Lesão,Parte Corpo Atingida,Sexo,Tipo do Acidente,UF Munic. Acidente,UF Munic. Empregador,Mês Acidente,Data Despacho Benefício,Data Acidente,Data Nascimento,Data Emissão CAT
40847,{ñ class},000000-Não Informado,S92.5 Frat de Outr Artelho,{ñ class},{ñ class},Auxílio Doenca por Acidente do Trabalho,{ñ class},{ñ,000000-Ignorado,{ñ class},{ñ class},Feminino,Ignorado,Alagoas,Zerado,2022/02,2022/04,10/02/2022,05/06/1979,00/00/0000
40848,{ñ class},000000-Não Informado,S36 Traum de Orgaos Intra-Abdominais,{ñ class},{ñ class},Auxílio Doenca por Acidente do Trabalho,{ñ class},{ñ,000000-Ignorado,{ñ class},{ñ class},Masculino,Ignorado,Alagoas,Zerado,2022/02,2022/03,23/02/2022,10/11/1979,00/00/0000
40849,{ñ class},000000-Não Informado,S52.5 Frat da Extremidade Distal do Radio,{ñ class},{ñ class},Auxílio Doenca por Acidente do Trabalho,{ñ class},{ñ,000000-Ignorado,{ñ class},{ñ class},Masculino,Ignorado,Alagoas,Zerado,2022/02,2022/05,11/02/2022,09/01/1972,00/00/0000
40850,{ñ class},000000-Não Informado,S62.6 Frat de Outr Dedos,{ñ class},{ñ class},Auxílio Doenca por Acidente do Trabalho,{ñ class},{ñ,000000-Ignorado,{ñ class},{ñ class},Masculino,Ignorado,Alagoas,Zerado,2022/02,2022/07,06/02/2022,08/09/1981,00/00/0000
40851,{ñ class},000000-Não Informado,G56.0 Sindr do Tunel do Carpo,{ñ class},{ñ class},Auxílio Doenca por Acidente do Trabalho,{ñ class},{ñ,000000-Ignorado,{ñ class},{ñ class},Feminino,Ignorado,Sergipe,Zerado,2022/02,2022/05,10/02/2022,29/04/1972,00/00/0000


In [ ]:
# Faz um resumo estatístico do dataset (média, desvio padrão, mínimo, máximo e os quartis)
dataset.describe()

,Agente Causador Acidente,CBO,CID-10,CNAE2.0 Empregador,Emitente CAT,Espécie do benefício,Filiação Segurado,Indica Óbito Acidente,Munic Empr,Natureza da Lesão,Parte Corpo Atingida,Sexo,Tipo do Acidente,UF Munic. Acidente,UF Munic. Empregador,Mês Acidente,Data Despacho Benefício,Data Acidente,Data Nascimento,Data Emissão CAT
count,40852,40852,40852,40852,40852,40852,40852,40852,40852,40852,40852,40852,40852,40852,40852,40852,40852,40852,40852,40852
unique,242,862,1590,528,6,5,4,3,1379,30,46,4,4,17,28,2,8,59,13168,62
top,{ñ class},000000-Não Informado,S62.6 Frat de Outr Dedos,{ñ class},{ñ class},Auxílio Doenca por Acidente do Trabalho,{ñ class},{ñ,000000-Ignorado,{ñ class},{ñ class},Masculino,Ignorado,{ñ class},Zerado,2022/02,2022/03,28/02/2022,03/10/1995,00/00/0000
freq,29787,28718,2204,29210,28844,28659,28826,28717,29479,28717,28717,27382,28616,13295,29480,22961,13949,1531,13,28717


Vamos agora verificar se o dataset tem as classes balanceadas para que possamos tratar o desbalanceamento posteriormente, se necessário. Veremos que as classes 0 (não ocorrência de diabetes) e 1 (ocorrência de diabetes) estão desbalanceadas. Vamos guardar esta informação, pois possivelmente precisaremos realizar algum tipo de tratamento nas próximas etapas.

In [ ]:
# distribuição das classes
#print(dataset.groupby('class').size())

### 3.2. Visualizações Unimodais

Vamos criar agora um histograma para cada atributo do dataset. Veremos que os atributos age, pedi e test seguem uma distribuição exponencial, e que as colunas mass e press seguem uma distribuição aproximadamente normal.

In [ ]:
# Histograma
dataset.hist(figsize = (15,10))
plt.show()

ValueError: ignored

O Gráfico de Densidade, ou Density Plot, é bem parecido com o histograma, mas com uma visualização um pouco diferente. Com ele, pode ser mais fácil identificar a distribuição do atributos do dataset. Assim como fizemos com o histograma, vamos criar um density plot para cada atributo do dataset.

Veremos que muitos dos atributos têm uma distribuição distorcida. Uma transformação como a Box-Cox, que pode aproximar a distribuição de uma Normal, pode ser útil neste caso.

In [ ]:
# Density Plot
dataset.plot(kind = 'density', subplots = True, layout = (3,3), sharex = False, figsize = (15,10))
plt.show()

Vamos agora trabalhar com boxplots. No **boxblot**, a linha no centro (vermelha) representa o valor da mediana (segundo quartil ou p50). A linha abaixo é o 1o quartil (p25) e a linha acima o terceiro quartil (p75). O boxplot ajuda a ter uma ideia da dispersão dos dataset e os possíveis outliers.

*OBS: Se um ponto do dataset é muito distante da média (acima de 3 desvios padrão da média), pode ser considerado outlier.*

Nos gráficos bloxplot, veremos que a dispersão dos atributos do dataset é bem diferente.

In [ ]:
# Boxplot
dataset.plot(kind = 'box', subplots = True, layout = (3,3), sharex = False, sharey = False, figsize = (15,10))
plt.show()

### 3.3. Visualizações Multimodais

Ao visualizar as correlações entre os atributos através da matriz de correlação, perceberemos que parece haver alguma estrutura na ordem dos atributos. O azul ao redor da diagonal sugere que os atributos que estão próximos um do outro são geralmente mais correlacionados entre si. Os vermelhos também sugerem alguma correlação negativa moderada, a medida que os atributos 

Vamos agora verificar a covariância entre as variáveis numéricas do dataset. A **covariância** representa como duas variáveis numéricas estão relacionadas. Existem várias formas de calcular a correlação entre duas variáveis, como por exemplo, o coeficiente de correlação de Pearson, que pode ser:
* Próximo de -1 : há uma correlação negativa entre as variáveis, 
* Próximo de +1: há uma correlação positiva entre as variáveis. 
* 0: não há correlação entre as variáveis.

<i>OBS: Esta informação é relevante porque alguns algoritmos como regressão linear e regressão logística podem apresentar problemas de performance se houver atributos altamente correlacionados. Vale a pena consultar a documentação do algoritmo para verificar se algum tipo de tratamento de dataset é necessário.</i>

Falamos anteriormente da importância da correlação entre os atributos, e agora iremos visualizar esta informação em formato gráfico. A **matriz de correlação** exibe graficamente a correlação entre os atributos numéricos do dataset.estão mais distantes um do outro na ordenação. 

O código a seguir exibe a matriz de correlação.

In [ ]:
# Matriz de Correlação com Matplotlib Seaborn
sns.heatmap(dataset.corr(), annot=True, cmap='RdBu');

Por sua vez, o gráfico de dispersão (**scatter plot**) mostra o relacionamento entre duas variáveis. Vamos exibir um para cada par de atributos dos dataset, usando o Seaborn.

In [ ]:
# Scatter Plot com Seaborn - Variação 1

sns.pairplot(dataset)

In [ ]:
# Scatter Plot com Seaborn - Variação 2

sns.pairplot(dataset, hue = "class", height = 2.5);

## 4. Pré-Processamento de dados

Nesta etapa, poderíamos realizar diversas operações de preparação de dados, como por exemplo, tratamento de valores missings (faltantes), limpeza de dados, transformações como one-hot-encoding, seleção de características (feature selection), entre outras não mostradas neste notebook. Lembre-se de não criar uma versão padronizada/normalizada dos dados neste momento (apesar de serem operações de pré-processamento) para evitar o Data Leakage.

### 4.1. Tratamento de Missings e Limpeza

Sabemos que o datset Diabetes não tem missings aparentes, mas valores "0" que parecem ser missings. Vamos então fazer este tratamento e criar uma nova visão do nosso dataset.

In [ ]:
# verificando nulls no dataset
dataset.isnull().sum()

Agente  Causador  Acidente    0
CBO                           0
CID-10                        0
CNAE2.0 Empregador            0
Emitente CAT                  0
Espécie do benefício          0
Filiação Segurado             0
Indica Óbito Acidente         0
Munic Empr                    0
Natureza da Lesão             0
Parte Corpo Atingida          0
Sexo                          0
Tipo do Acidente              0
UF  Munic.  Acidente          0
UF Munic. Empregador          0
Mês Acidente                  0
Data Despacho Benefício       0
Data Acidente                 0
Data Nascimento               0
Data Emissão CAT              0
dtype: int64

In [ ]:
# salvando um NOVO dataset para tratamento de missings (cuidado para não sobrescrever o dataset original!)

# recuperando os nomes das colunas
col = list(dataset.columns)

# o novo dataset irá conter todas as colunas com exceção da última (classe)
atributos = dataset[col[0:-1]]

# substituindo os zeros por NaN
atributos.replace(0, np.nan, inplace=True)

# exibindo visualização matricial da nulidade do dataset
ms.matrix(atributos)

<ipython-input-62-6347d325c36d>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  atributos.replace(0, np.nan, inplace=True)


NameError: ignored

In [ ]:
# removendo as colunas 'skin' e 'test'
atributos.drop(['skin', 'test'], axis=1, inplace= True)

# exibindo visualização matricial da nulidade do dataset
ms.matrix(atributos)

In [ ]:
# substituindo os NaN de 'preg' por 0
atributos['preg'].fillna(0, inplace=True)

# substituindo os NaN de 'plas', 'pres'e 'mass' pela mediana da coluna
atributos['plas'].fillna(atributos['plas'].median(), inplace=True)
atributos['pres'].fillna(atributos['pres'].median(), inplace=True)
atributos['mass'].fillna(atributos['mass'].median(), inplace=True)

# exibindo visualização matricial da nulidade do dataset
ms.matrix(atributos)

In [ ]:
# Guardando o novo dataset para testes futuros
datasetSemMissings = atributos

# incluindo a coluna 'class' no novo dataset
datasetSemMissings['class'] = dataset['class']

# exibindo as primeiras linhas
datasetSemMissings.head()

### 4.2. Separação em conjunto de treino e conjunto de teste

É uma boa prática usar um conjunto de teste (na literatura também chamado de conjunto de validação), uma amostra dos dados que não será usada para a construção do modelo, mas somente no fim do projeto para confirmar a precisão do modelo final. É um teste que podemos usar para verificar o quão boa foi a construção do modelo, e para nos dar uma ideia de como o modelo irá performar nas estimativas em dados não vistos. Usaremos 80% do conjunto de dados para modelagem e guardaremos 20% para teste, usando a estratégia train-test-split, já explicada anteriormente. Primeiramente, iremos sinalizar quais são as colunas de atributos (X - 0 a 7) e qual é a coluna das classes (Y - 8). Em seguida, especificaremos o tamanho do conjunto de teste desejado e uma semente (para garantir a reprodutibilidade dos resultados). Finalmente, faremos a separação dos conjuntos de treino e teste através do comando train_test_split, que retornará 4 estruturas de dados: os atributos e classes para o conjunto de teste e os atributos e classes para o conjunto de treino.


In [ ]:
test_size = 0.20
seed = 7

# Separação em conjuntos de treino e teste (dataset original)
array = dataset.values
X = array[:,0:8]
y = array[:,8]
#X_train, X_test, y_train, y_test = train_test_split(X, y,
#    test_size=test_size, shuffle=True, random_state=seed) # sem estratificação
X_train, X_test, y_train, y_test = train_test_split(X, y,
    test_size=test_size, shuffle=True, random_state=seed, stratify=y) # com estratificação

In [ ]:
# Separação em conjuntos de treino e teste (dataset sem missings - 2 colunas a menos!)
array = datasetSemMissings.values
X_sm = array[:,0:6]
y_sm = array[:,6]
#X_train_sm, X_test_sm, y_train_sm, y_test_sm = train_test_split(X_sm, y_sm,
#    test_size=test_size, shuffle=True, random_state=seed) # sem estratificação
X_train_sm, X_test_sm, y_train_sm, y_test_sm = train_test_split(X_sm, y_sm,
    test_size=test_size, shuffle=True, random_state=seed, stratify=y_sm) # com estratificação